## Welcome to PADL 
- A python module supporting PADL privacy-preserving distributed ledger system.

The tutorial is split into sections listed below:
1) MakeLedger
2) Bank
3) Auditing 

Pre-requisites:
1) Python
2) ZKPB module listed under the python search path
3) Rust Compilation Tools, rustup (if ZKPB module is not compiled)
4) Jupyter Notebook and its environment, ie ipykernel

***

## Common Functionality

Let's start by importing some basic functionalities.


In [5]:
from ..pyledger.ledger import MakeLedger, BankCommunication, Bank, Auditing 


ImportError: attempted relative import with no known parent package

If the example return error, try to restart the notebook/kernel to reset the state.

***

### Ledger, Bank and Transaction

### MakeLedger 

Ledger:
<!-- - `register_bank(bank.pk, bank.address)` registers a bank onto the ledger. NOTE: creating bank then register_bank is not the correct flow. -->
- `MakeLedger(comm,addr)` constructs a Ledger object that closely matches the Ledger described in PADL.
- `register_new_bank(args)` constructs a new bank instance and register it onto the ledger.
- `get_set_n_banks()` gives number of enrolled bank.
- `register_zero_line(cell)` override the initial cell. (Should not be called from outside the class)
- `retrieve_txs()` return the distributed txs (trasactions that are yet to be appended to ledger).

Trasnaction:
- `populate_tx(tx)` performs necessary state maangement for offline mode.
- `push_tx(tx/txs, verify=True)` pushs transaction onto ledger (with verification [audit_tx] by default).
- `audit_tx(tx/txs)` performs audit verification (proofs of asset, consistency, etc.) on the transaction.
- `to_json(tx)` serializes the trasnaction into JSON format.
-  `txs_from_json(txs_json)` deserializes JSON formatted transaction into Cell(Transaction) object. Another variant is `tx_from_json(tx_json)` that is used for deserializing from single-line tx.
- `txs3d_from_json(txs3d_json)` deserializes JSON formmated transaction into assets.
- `ledger.compute_sum_commits_tokens()` computes commit token sum that is needed for audit purposes. [It is normally used together with audit class. See Auditing section below.]
<!-- - `create_initial_cell_from_asset_vals(v0, pk)` create a zero_line cell object based on provided initial v0 value and asset party public key which then can be used to initialize ledger.  -->

### Bank

- Bank object represents a bank entity described in PADL.
- To create a bank object, run `Bank(api.ledger, v0=[0], type=None, address=api.local_host, port=0, serialise=False)` whereby 
    - ledger: the ledger the bank should be communcating to
    - v0 : initial asset value list
    - address : bank digital address 
    - type : asset type 
    - port : network port for the given address
    - serialize : save to file 
- `bank.add_asset(value, asset_type)` adds asset under the bank account.
- `bank.create_asset_tx(txs, n_banks, bank_public_keys)` creates new trasnaction based on the provided txs, n_banks and pks.


In [ ]:
# The example here make use of most of the functions described above.
# It start by creating dummy communication channel for local debugging purpose.
# Next, a ledger is created, and then new banks are added to the ledger.
# Finally, a number of transactions are created and appended to the ledger. 

n_banks = 2

dummy_communication = BankCommunication()
ledger = MakeLedger(dummy_communication)

banks = []

for b in range(n_banks):
    bank = ledger.register_new_bank(v0=[1000], types={0: 'xcoin'})
    print(bank.secret_balance_book)
    banks.append(bank)

print("Number of Bank:",ledger.get_set_n_banks())
assert ledger.get_set_n_banks() == n_banks, "Numberof enrolled bank does not match"

for i in range(2):
    bi = i % n_banks
    print('Bank {} makes Tx no. {}'.format(bi, i))
    distributed_tx = banks[bi].create_rand_tx(n_banks, ledger.pub_keys)
    ledger.populate_tx(distributed_tx)
    ledger.push_tx(distributed_tx)
    for i in range(0,len(banks)):
        print((banks[i].secret_balance_book[0]))
    distributed_tx = banks[bi].create_rand_tx(n_banks, ledger.pub_keys)
    ledger.populate_tx(distributed_tx)
    ledger.audit_tx(distributed_tx)
    ledger.push_tx(distributed_tx)
    tx_json = MakeLedger.to_json(distributed_tx)
    print("Transaction:" ,MakeLedger.to_json(distributed_tx))
    recons_tx = MakeLedger.txs_from_json(tx_json)
    

assert ledger.retrieve_txs()==[],"Pending trasnactions should be empty." #Expected empty as the pending transaction had been processed.
print("test - adding asset")        
# Dynamically adding new asset
init_cell_asset=banks[0].add_asset(2000, asset_type="zcoin")
ledger.register_new_asset(init_cell_asset,0)
tx = [0] * n_banks
tx[0] = -100
tx[1] = 100
txs = [tx.copy() for _ in range(2)]
distributed_tx = banks[0].create_asset_tx(txs, n_banks, ledger.pub_keys)
ledger.populate_tx(distributed_tx)
ledger.push_tx(distributed_tx)

***
### Auditing (Generation and Verification of Proof)
<!-- - `audit_asset_balance(ledger, bank_id, asset)` performs the same check as above but generate the necessary proof inside the function.  This function reference a non-existence function get_asset_balance_proof()-->
- `verify_asset_balance(sum, generator_gh, proof, sval, bank_id, asset_id)` performs asset balance check on the chosen bank asset.
- `valdiate_proof_of_ext_consistency`, `valdiate_proof_of_consistency`, `valdiate_proof_of_asset`, `verify_value_eq_cm` is called during audit_tx to audit transaction. [See above for example of it].
- `validate_proof_of_balance(tx)` verifies transaction satisfy balance requirement.
- `valdiate_proof_of_ratio_asset` verifies the claimed asset liquidity which is generated using `generate_asset_ratio_proof(asset_id, ratio_n, ratio_d)`.

In [ ]:
# Continuing from the example above, here we are going to generate proof for auditing making use of the function mentioned in the Markdown section above.
# ------------------- test auditing of asset --------------
# generate balance proof
scst = ledger.compute_sum_commits_tokens()
from pyledger.Proof_Generation import ProofGenerator
for asset_id in range(1):
    pr, sval = (ProofGenerator().generate_asset_balance_proof(scst, asset_id, banks[0], ledger))
    assert Auditing.verify_asset_balance(scst, ledger.gh, pr, sval, 0, asset_id), "auditing is wrong"
    sval[asset_id] = sval[asset_id] + 5
    assert not Auditing.verify_asset_balance(scst, ledger.gh, pr, sval, 0, asset_id), "auditing is wrong"
print('audited balance')

print(banks[0].get_balances_from_state(ledger))
# generate ratio asset threshold proof (liquidity of 1/3 threshold)
proof=banks[0].generate_asset_ratio_proof(asset=0, n=1,d=3)
Auditing.valdiate_proof_of_ratio_asset(ledger,proof, asset=0, n=1,d=3)

# Making use of the distributed tx before
assert Auditing.validate_proof_of_balance(distributed_tx[0]), "Proof of balance does not agree."